# Lecture 2: Introduction to Feature Engineering

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [2]:
articles = pd.read_csv('../data/articles.csv')
customers = pd.read_csv('../data/customers.csv')
sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

## The H&M Dataset

In [3]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [4]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [6]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [7]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05


In [8]:
articles_sample = pd.read_csv('../data/articles_sample01.csv.gz')
customers_sample = pd.read_csv('../data/customers_sample01.csv.gz')
transactions_sample = pd.read_csv('../data/transactions_sample01.csv.gz')

In [9]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             1371 non-null   object 
 1   FN                      452 non-null    float64
 2   Active                  431 non-null    float64
 3   club_member_status      1364 non-null   object 
 4   fashion_news_frequency  1356 non-null   object 
 5   age                     1354 non-null   float64
 6   postal_code             1371 non-null   object 
dtypes: float64(3), object(4)
memory usage: 75.1+ KB


In [10]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31021 entries, 0 to 31020
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   t_dat             31021 non-null  object 
 1   customer_id       31021 non-null  object 
 2   article_id        31021 non-null  int64  
 3   price             31021 non-null  float64
 4   sales_channel_id  31021 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ MB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
To obtain the 12 items a user is most likely to buy, we then make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [11]:
# If you'd rather use a sample. Uncomment the following code:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [12]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [13]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,3eb5d8709fd95ea7d4ab674b55e1cad5311a5b50181240...,597763001,0.010153,2,1
1,2018-09-20,3eb5d8709fd95ea7d4ab674b55e1cad5311a5b50181240...,597763001,0.010153,2,1
2,2018-09-20,5c5dc9eaa407df983b3de8878649669b91cdf8c6694076...,637515001,0.033881,1,1
3,2018-09-20,5c5dc9eaa407df983b3de8878649669b91cdf8c6694076...,561570001,0.033881,1,1
4,2018-09-20,62abe5160f2c34a91debc9a4b2a7f055fd5627f209ee43...,573152003,0.025407,2,1


In [77]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [26]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,3eb5d8709fd95ea7d4ab674b55e1cad5311a5b50181240...,597763001,0.010153,2,1
1,2018-09-20,3eb5d8709fd95ea7d4ab674b55e1cad5311a5b50181240...,597763001,0.010153,2,1
2,2018-09-20,5c5dc9eaa407df983b3de8878649669b91cdf8c6694076...,637515001,0.033881,1,1
3,2018-09-20,5c5dc9eaa407df983b3de8878649669b91cdf8c6694076...,561570001,0.033881,1,1
4,2018-09-20,62abe5160f2c34a91debc9a4b2a7f055fd5627f209ee43...,573152003,0.025407,2,1


In [59]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [60]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

31021


In [62]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [70]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [72]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2020-06-27,98b74fc1e8272626ba7f0398911b981fc36da899fc2436...,802864001,0.050831,1,0
1,2019-06-19,e72055f1a6cb552fca32646e092a7da22ef43c570d262e...,572400022,0.033881,2,0
2,2018-12-14,da4200eceac8a06b4647b944b00149b2c79476a625b2a8...,572128001,0.008458,2,0
3,2020-07-18,03c564dd3f029dd04d8d365a2ff7f77e6f8f77c308a5fd...,679709002,0.005407,2,0
4,2019-02-10,b9fc9ede777baa3cc075b21c084bbb0ded5a72da95a869...,714790002,0.043203,1,0


In [13]:
# Get real article transactions:
# transaction_articles = transactions.drop(labels=['t_dat','customer_id', 'ordered', 'sales_channel_id'], axis=1)
# transaction_articles = transaction_articles.drop_duplicates(subset=['article_id'])
# transaction_articles.head()

# customer_ids = customers['customer_id'].unique().tolist()
# rand_customers = choices(customer_ids, k=len(transactions))
# article_ids = articles['article_id'].unique().tolist()
# rand_articles = choices(article_ids, k=len(transactions))
# rand_channels = choices([1, 2], k=len(transactions))
# Inserting the random users, articles and channels in a transactions_rand dataframe
# transactions_rand = transactions.copy()
# transactions_rand.drop(['price'], axis=1, inplace=True)
# transactions_rand['customer_id'] = rand_customers 
# transactions_rand['article_id'] = rand_articles
# transactions_rand['sales_channel_id'] = rand_channels
# # Merge the random users with actual pricing from random articles, takes a while to run (merging 38M with 3.8M rows):
# transactions_rand = transactions_rand.merge(transaction_articles, on='article_id', how='left')
# transactions_rand['ordered'] = 0

In [73]:
neg_transactions.shape

(34123, 6)

In [83]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [84]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [85]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62042 entries, 0 to 62041
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   t_dat                         62042 non-null  object 
 1   customer_id                   62042 non-null  object 
 2   article_id                    62042 non-null  object 
 3   price                         62042 non-null  object 
 4   sales_channel_id              62042 non-null  object 
 5   ordered                       62042 non-null  object 
 6   FN                            22283 non-null  float64
 7   Active                        21553 non-null  float64
 8   club_member_status            61637 non-null  object 
 9   fashion_news_frequency        61581 non-null  object 
 10  age                           61534 non-null  float64
 11  postal_code                   62042 non-null  object 
 12  product_code                  62042 non-null  int64  
 13  p

### Basic Preprocessing
Some very basic preprocessing.

In [86]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,3eb5d8709fd95ea7d4ab674b55e1cad5311a5b50181240...,54.0,597763001,2,0.010153,1
1,3eb5d8709fd95ea7d4ab674b55e1cad5311a5b50181240...,54.0,597763001,2,0.010153,1
2,9123aa00cf7dbe63b2d8f77fc92982f932986148394b78...,25.0,597763001,2,0.013542,1
3,9e0498586cb5d8e2a7304af71c63c6d73f5a742945bec4...,76.0,597763001,2,0.010153,0
4,a83261106bbfdbee89f66af3b5151f2833602420439c1d...,25.0,597763001,1,0.010153,0


In [87]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [88]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [89]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['009d1f01410c945ed5f25c733f2c8261c5f17800fab8b4c8875efee404ca59fc'],
      dtype=object)

In [90]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,328,54.0,2629,2,0.010153,1
1,328,54.0,2629,2,0.010153,1
2,780,25.0,2629,2,0.013542,1
3,839,76.0,2629,2,0.010153,0
4,889,25.0,2629,1,0.010153,0


In [91]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [92]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [93]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,328,54.0,2629,0.010153,1,0,1
1,328,54.0,2629,0.010153,1,0,1
2,780,25.0,2629,0.013542,1,0,1
3,839,76.0,2629,0.010153,0,0,1
4,889,25.0,2629,0.010153,0,1,0


In [95]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [96]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
26083,611,25.0,11521,0.032525,0,1
19127,1076,20.0,4670,0.020322,0,1
54475,1039,23.0,14566,0.013542,1,0
24141,1347,26.0,1083,0.011610,1,0
14581,529,28.0,4137,0.035576,0,1


In [97]:
y_train.head()

26083    0
19127    0
54475    0
24141    1
14581    1
Name: ordered, dtype: int64

## Baseline Model

In [98]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [99]:
baseline.predict_proba(X_test)

array([[0.64166579, 0.35833421],
       [0.39650711, 0.60349289],
       [0.60826657, 0.39173343],
       ...,
       [0.42729266, 0.57270734],
       [0.39363852, 0.60636148],
       [0.61942444, 0.38057556]])

In [100]:
y_test

24106    0
9620     1
22657    1
53613    1
59312    0
        ..
55453    0
10675    0
56215    1
40691    1
54517    0
Name: ordered, Length: 6205, dtype: int64

In [101]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.6085414987912974

In [103]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.64      0.49      0.56      3080
           1       0.59      0.72      0.65      3125

    accuracy                           0.61      6205
   macro avg       0.61      0.61      0.60      6205
weighted avg       0.61      0.61      0.60      6205



In [104]:
predictions

array([0, 1, 0, ..., 1, 1, 0])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).
